In [14]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time 

# 크롬 창을 띄우지 않고 실행하는 옵션
#chrome_option = webdriver.ChromeOptions()
#chrome_option.add_argument('headless')
#chrome_option.add_argument("disable-gpu")

In [15]:
#함수화 - 날짜 입력시 5개의 상위 검색결과 가져오기

def hawaii2(startday, endday) :
    
    url_start = "https://beta-flight.naver.com/flights/international/ICN-HNL-" 
    date1 = "2021" +str(startday) 
    url_middle = "/HNL-ICN-"
    date2 = "2021" + str(endday) 
    url_last = "?adult=1&fareType=Y"
    url = url_start+date1+url_middle+date2+url_last

    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)

    time.sleep(20)
    #데이터 불러오는 시간
    #implicitly_wait 사용?

    price_xpath = '//*[@id="__next"]/div/div[1]/div[4]/div/div[3]/div/div/div[2]/div/div/div/b/i'
    go_xpath = '//*[@id="__next"]/div/div[1]/div[4]/div/div[3]/div/div/div[1]/div/div[2]/div[1]/i'
    back_xpath = '//*[@id="__next"]/div/div[1]/div[4]/div/div[3]/div/div/div[1]/div/div[2]/div[2]/i'

    sel_price = driver.find_elements_by_xpath(price_xpath)
    go_times = driver.find_elements_by_xpath(go_xpath)
    back_times = driver.find_elements_by_xpath(back_xpath)

    day_rank5 = []
    for i in range(5) :
        key = []
        key.append(go_times[i].text[6:])
        key.append(back_times[i].text[6:])
        key.append(sel_price[i].text)
        #print(key)
        day_rank5.append(key)
    return day_rank5


In [16]:
# 12월 5일 출발 예시

# 상위 5개의 결과의 비행시간 돌려주는 함수
def find_time(date1,date2) :
    a = hawaii2(date1,date2)
    timeset = []
    for i in range(5) :
        hour = int(a[i][0][0:2])+int(a[i][1][0:2])
        minute = int(a[i][0][5:7])+int(a[i][1][5:7])
        total_time = hour*60 + minute
        
        timeset.append(total_time)
        
    return timeset

In [17]:
# 상위 5개의 결과의 비용 돌려주는 함수
def find_price(date1,date2) :
    a = hawaii2(date1,date2)
    priceset = []
    for i in range(5) :
        price_str = a[i][2]
        total_price = int(price_str[0:3])*1000 + int(price_str[4:])
    
        priceset.append(total_price)
        
    return priceset

In [18]:
# 상위 5개의 결과의 튜플(비용, 비행시간) 돌려주는 함수
def find_gasungbee(date1,date2) :
    a = hawaii2(date1,date2)
    gasungbee = []
    for i in range(5) :
        hour = int(a[i][0][0:2])+int(a[i][1][0:2])
        minute = int(a[i][0][5:7])+int(a[i][1][5:7])
        total_time = hour*60 + minute
        price_str = a[i][2]
        total_price = int(price_str[0:3])*1000 + int(price_str[4:])
    
        ratio = (total_price , total_time)
        gasungbee.append(ratio)
        
    return gasungbee

In [19]:
# 파일 생성
import pandas as pd

#dict_dat = {"1205출발,1210도착" : find_gasungbee(1205,1210),
#           "1205출발,1211도착" : find_gasungbee(1205,1211),
#           "1205출발,1212도착" : find_gasungbee(1205,1212),...}
dict_dat = {}
for i in range(10,14,1) :
    date = 1200+i
    key = str(date)+"도착"
    dict_dat[key] = find_gasungbee(1205,date)
    
dat = pd.DataFrame(dict_dat)

ValueError: invalid literal for int() with base 10: '1,0'

In [12]:
dat

,1210도착,1211도착,1212도착,1213도착
0,"(852500, 1850)","(852500, 1850)","(852500, 1850)","(852500, 1850)"
1,"(880500, 1800)","(880500, 1800)","(880500, 1800)","(885500, 2160)"
2,"(885500, 2160)","(885500, 2160)","(885500, 2160)","(890200, 2795)"
3,"(890200, 2745)","(890200, 2215)","(890200, 2745)","(968400, 2667)"
4,"(901100, 2478)","(902500, 3563)","(901100, 2392)","(977000, 3072)"


In [10]:
dat.to_csv("flight01.csv", index=False)

In [11]:
dat.to_excel("flight01.xlsx", index = False)